# Importing the data

In [64]:
import pandas as pd

In [65]:
df_resume_1=pd.read_csv('raw_data/ResumeDataSet-1.csv')
df_resume_2=pd.read_csv('raw_data/ResumeDataSet-2.csv')
df_resume_3=pd.read_csv('raw_data/ResumeDataSet-3.csv')

# Création du df_resume


**Objectif** : stocker toutes les données dans un seul DataFrame avec deux features : `Resume` et `Category`

In [66]:
df_resume=pd.DataFrame()

## Préparation de `df_resume_1`

In [67]:
df_resume_1.head(3)

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."


In [68]:
#switcher les deux colonnes Category et Resume
df_resume_1 = df_resume_1[['Resume', 'Category']]

In [69]:
#importer df_resume_1 dans df_esume
df_resume=df_resume_1

In [70]:
df_resume

,Resume,Category
0,Skills * Programming Languages: Python (pandas...,Data Science
1,Education Details \r\nMay 2013 to May 2017 B.E...,Data Science
2,"Areas of Interest Deep Learning, Control Syste...",Data Science
3,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,Data Science
4,"Education Details \r\n MCA YMCAUST, Faridab...",Data Science
...,...,...
957,Computer Skills: â¢ Proficient in MS office (...,Testing
958,â Willingness to accept the challenges. â ...,Testing
959,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...",Testing
960,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...,Testing


## Préparation de `df_resume_2`

In [71]:
df_resume_2.head(3)

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [72]:
df_resume_2.loc[100]['Resume_str']

'         HR BENEFITS/LEAVE COORDINATOR       Summary    13 years of Human Resources experience and 27 years of administrative experience working in various settings\n*Professional, detail-oriented, excellent time-management skills, team player, open and honest communicator, effective at maintaining confidentiality and multi-tasker.      Skills        Proficient in all Microsoft applications and HRIS systems (Banner, PeopleAdmin, Personnel Management Information (PMIS) System and Benefits Eligibility System (BES), VRS Navigator (VNAV))            Experience     December 2016   to   Current     Company Name   City  ,   State     HR Benefits/Leave Coordinator        Provide consultation and assistance for all employee benefits.  Assist Retirement/Benefits Manager in counseling employees on retirements as well as generate retirement calculations.  Assist Retirement/Benefits Manager with retirement trainings.  Promote and coordinate all aspects of the Highlander Wellness Watch program.  Ad

🙌 **Warning** : the category word is probably already present in the resume. There is a risk of dataleackage. 



In [73]:
df_resume_2=df_resume_2[['Resume_str', 'Category']].rename(columns={'Resume_str': 'Resume'})

In [74]:
df_resume_2['Resume'] = df_resume_2.apply(lambda row: row['Resume'].replace(row['Category'], '') if pd.notnull(row['Category']) else row['Resume'], axis=1)

In [75]:
df_resume_2

,Resume,Category
0,ADMINISTRATOR/MARKETING ASSOCIATE\n\...,HR
1,"SPECIALIST, US OPERATIONS Sum...",HR
2,DIRECTOR Summary Over 20 ...,HR
3,SPECIALIST Summary Dedicate...,HR
4,MANAGER Skill Highlights ...,HR
...,...,...
2479,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,AVIATION
2480,"GOVERNMENT RELATIONS, COMMUNICATIONS ...",AVIATION
2481,GEEK SQUAD AGENT Professional...,AVIATION
2482,PROGRAM DIRECTOR / OFFICE MANAGER ...,AVIATION


In [76]:
df_resume=pd.merge(df_resume, df_resume_2, on=['Resume', 'Category'], how='outer')

In [77]:
df_resume

,Resume,Category
0,Skills * Programming Languages: Python (pandas...,Data Science
1,Skills * Programming Languages: Python (pandas...,Data Science
2,Skills * Programming Languages: Python (pandas...,Data Science
3,Skills * Programming Languages: Python (pandas...,Data Science
4,Education Details \r\nMay 2013 to May 2017 B.E...,Data Science
...,...,...
3441,ADVANCED LEVEL WHEELED VEHICLE MECHAN...,AVIATION
3442,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,AVIATION
3443,"GOVERNMENT RELATIONS, COMMUNICATIONS ...",AVIATION
3444,GEEK SQUAD AGENT Professional...,AVIATION


## Préparation de `df_resume_3`

In [78]:
df_resume_3.head()

,worker_id,task_id,task_response_id,ID,Resume_str,Resume_html,What is their work experience level?,What best describes their education level?,What industry and sub-sector (most recent work) best describes this resume?,"You selected ""N/A: Other Industry / Other Category"".....write in the best description (1-3 words) of what industry should be?"
0,J6TM2D4MANTG,da70f4c8-a26f-4330-993f-0c357de0225a,fc506e4e-99f9-435a-b571-bd1af0d5bcda,46514339,JOBS COORDINATOR / ESTIMATOR Professional Summ...,"<div class=""fontsize fontface vmargins hmargin...",Mid-level (5-10 yr),"Some college or university: trade school, asso...",NaN,NaN
1,YYCA2DXN94NG,0109f1e1-2d5e-40eb-8157-7641bd3bb97c,b810e5a3-f05d-4537-80c1-36350865f222,18997135,COURT APPOINTED SPECIAL ADVOCATE Summary Finan...,"<div class=""fontsize fontface vmargins hmargin...",Mid-level (5-10 yr),Bachelor's,N/A: Other Industry / Other Category,Law
2,J6TM2D4MANTG,543330a3-e182-4f05-87a9-56d965a92d2c,83e9c697-647d-4baf-a0ed-00f2133a0169,12092347,SALES ASSOCIATE Professional Summary Jack of a...,"<div class=""fontsize fontface vmargins hmargin...",Early career (2-5 yr),No mention of college or post high-school educ...,Retail / Manager or Store Keeper (includes cle...,NaN
3,PAEJPZXXTQ2M,0d4031c9-63e2-48d6-909e-23e885397966,b15a3a08-24f3-4a35-b0af-6baff37995c5,33627938,PERSONNEL SECURITY SPECIALIST Summary Dedicate...,"<div class=""fontsize fontface vmargins hmargin...","Senior (+10 yr, not executive)",Master's,N/A: Other Industry / Other Category,Safety and Security
4,6XA9X4RD9RAY,d6e3b6ca-8233-4433-878e-148eb79b6e2f,93a849e6-f2fd-430d-9fa5-09aa750b7115,27176039,ROUTE MANAGER Summary To obtaining a position ...,"<div class=""fontsize fontface vmargins hmargin...","Senior (+10 yr, not executive)","Some college or university: trade school, asso...",Retail / Manager or Store Keeper (includes cle...,NaN


In [79]:
columns=df_resume_3.columns

In [80]:
new_columns=[columns[4]]

In [81]:
df_resume_3=df_resume_3[new_columns]

In [82]:
df_resume_3=df_resume_3.rename(columns={'Resume_str': 'Resume'})

In [83]:
import re

# fonction qui extrait la première séquence de mots en majuscules`
def extract_cat(text):
    matches = re.findall(r'\b[A-Z&/][A-Z&/\s,]*[A-Z&/]\b', text)
    if matches:
        return matches[0]
    else:
        return None


In [84]:
df_resume_3['Category']= df_resume_3['Resume'].apply(extract_cat)
df_resume_3['Resume'] = df_resume_3.apply(lambda row: row['Resume'].replace(row['Category'], '') if pd.notnull(row['Category']) else row['Resume'], axis=1)

df_resume_3

,Resume,Category
0,Professional Summary [Job Title] with managem...,JOBS COORDINATOR / ESTIMATOR
1,Summary Financial professional who is a self-...,COURT APPOINTED SPECIAL ADVOCATE
2,Professional Summary Jack of all trades who e...,SALES ASSOCIATE
3,Summary Dedicated Security Specialist profess...,PERSONNEL SECURITY SPECIALIST
4,Summary To obtaining a position in a company ...,ROUTE MANAGER
...,...,...
570,Summary Seasoned Agriculture Teacher with mor...,ADULT EDUCATION INSTRUCTOR
571,Summary Dependable and resourceful Instructor...,INSTRUCTIONAL DESIGNER
572,Summary Candidate with a fifteen months exper...,ENGINEERING INTERN
573,Summary I have recently completed five years ...,QUALITY ASSURANCE ADVOCATE


In [85]:
df_resume=pd.merge(df_resume, df_resume_3, on=['Resume', 'Category'], how='outer')

In [86]:
df_resume

,Resume,Category
0,Skills * Programming Languages: Python (pandas...,Data Science
1,Skills * Programming Languages: Python (pandas...,Data Science
2,Skills * Programming Languages: Python (pandas...,Data Science
3,Skills * Programming Languages: Python (pandas...,Data Science
4,Education Details \r\nMay 2013 to May 2017 B.E...,Data Science
...,...,...
4016,Summary Seasoned Agriculture Teacher with mor...,ADULT EDUCATION INSTRUCTOR
4017,Summary Dependable and resourceful Instructor...,INSTRUCTIONAL DESIGNER
4018,Summary Candidate with a fifteen months exper...,ENGINEERING INTERN
4019,Summary I have recently completed five years ...,QUALITY ASSURANCE ADVOCATE


In [87]:
df_resume['Category'].value_counts()

HR                                      156
INFORMATION-TECHNOLOGY                  120
BUSINESS-DEVELOPMENT                    120
CHEF                                    118
ADVOCATE                                118
                                       ... 
HR CONSULTANT                             1
BILINGUAL DOMESTIC VIOLENCE ADVOCATE      1
BENEFIT ADVOCATE SUPERVISOR               1
SENIOR ENGINEERING PROGRAM MANAGER        1
PROGRAM MANAGER & DESIGNER                1
Name: Category, Length: 452, dtype: int64

In [88]:
df_resume['Resume_Length'] = df_resume['Resume'].apply(lambda x: len(str(x).split()))


In [89]:
df_resume.sort_values(by='Resume_Length',ascending=False)

,Resume,Category,Resume_Length
3073,BARTENDER Experience B...,PUBLIC-RELATIONS,5190
2328,Credentials National Regis...,CHEF,5079
2969,PROJECT MANGER/LEAD SUPER P...,CONSTRUCTION,3994
2779,Summary Financial Acco...,ACCOUNTANT,3458
2220,"CHIEF SYSTEM ARCHITECT, SVP SYSTEM IN...",DIGITAL-MEDIA,3301
...,...,...,...
60,Education Details \r\n BBA lovely profession...,HR,19
61,Education Details \r\n BBA lovely profession...,HR,19
63,Education Details \r\n BBA lovely profession...,HR,19
62,Education Details \r\n BBA lovely profession...,HR,19


# Dummy models

In [90]:
#display the score
resuls_dict = {'model': [], 'score': []}
results_df = pd.DataFrame(resuls_dict)

## DummyClassifier from sklearn

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [92]:
df = df_resume
df = df.dropna()

# Dictionnaire pour stocker les modèles avec différentes stratégies
dummy_models = {}
dummy_strategies = ['stratified', 'most_frequent', 'uniform', 'prior']

# Division en deux set d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df['Resume'], df['Category'], test_size=0.2, random_state=42)


for strategy in dummy_strategies:
    dummy_model = DummyClassifier(strategy=strategy, random_state=42)
    dummy_models[strategy] = dummy_model

# Entraînez et évaluez les modèles
for strategy, model in dummy_models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Accuracy for strategy '{strategy}': {accuracy:.2f}")
    new_line = {'model': f"DummyClassifier with '{strategy}' strategy" , 'score': round(accuracy,2)}
    results_df = results_df.append(new_line, ignore_index=True)

Accuracy for strategy 'stratified': 0.03
Accuracy for strategy 'most_frequent': 0.05
Accuracy for strategy 'uniform': 0.00
Accuracy for strategy 'prior': 0.05


/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_37208/258395860.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_line, ignore_index=True)
/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_37208/258395860.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_line, ignore_index=True)
/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_37208/258395860.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_line, ignore_index=True)
/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_37208/258395860.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in

## Logistic Regression Model TF-IDF 

In [93]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

df_resume_cleaned = df_resume.dropna(subset=['Resume', 'Category'])

X = df_resume_cleaned['Resume']
y = df_resume_cleaned['Category']

# Création d'ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création d'une représentation TF-IDF des textes
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Entraînement
logistic_regression_model = LogisticRegression(max_iter=500)
logistic_regression_model.fit(X_train_tfidf, y_train)

# Predict
predictions = logistic_regression_model.predict(X_test_tfidf)

# évalation
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

new_line = {'model': "LogisticRegression with TF-IDF" , 'score': round(accuracy,2)}
results_df = results_df.append(new_line, ignore_index=True)

Accuracy: 0.6069651741293532
Classification Report:
                                                                precision    recall  f1-score   support

                                            ACCOUNT EXECUTIVE       0.00      0.00      0.00         1
                                                   ACCOUNTANT       0.58      0.88      0.70        25
                                          ACCOUNTS PAYABLE AP       0.00      0.00      0.00         1
                      ADMINISTRATOR OF INFORMATION TECHNOLOGY       0.00      0.00      0.00         1
                                    ADVENTURE RUN COORDINATOR       0.00      0.00      0.00         1
                                                     ADVOCATE       0.43      0.50      0.46        32
                                                  AGRICULTURE       0.60      0.23      0.33        13
                                               AIRCRAFT SALES       0.00      0.00      0.00         1
                   

/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter t

## Text Classification with TF-IDF and Random Forest Model

In [94]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Removing the Nan
df_resume_cleaned = df_resume.dropna(subset=['Resume', 'Category'])

X = df_resume_cleaned['Resume']
y = df_resume_cleaned['Category']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer une représentation TF-IDF des textes
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Entraîner un modèle de forêt d'arbres décisionnels
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train_tfidf, y_train)

# Faire des prédictions sur l'ensemble de test
predictions = random_forest_model.predict(X_test_tfidf)

# Évaluer la performance du modèle
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
#print("Classification Report:\n", report)

new_line = {'model': "RandomForestClassifier with TF-IDF" , 'score': round(accuracy,2)}
results_df = results_df.append(new_line, ignore_index=True)


Accuracy: 0.5771144278606966


/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to contro

## Naive Baye Model


In [95]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn import metrics

# Removing the Nan
df_resume_cleaned = df_resume.dropna(subset=['Resume', 'Category'])

X = df_resume_cleaned['Resume']
y = df_resume_cleaned['Category']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer le modèle Naive Bayes
model = make_pipeline(CountVectorizer(), MultinomialNB())

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)

# Prédire les étiquettes sur les données de test
y_pred = model.predict(X_test)

# Afficher les performances du modèle
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", metrics.classification_report(y_test, y_pred))

new_line = {'model': "Naive Baye Model" , 'score': round(accuracy,2)}
results_df = results_df.append(new_line, ignore_index=True)

Accuracy: 0.5534825870646766

Classification Report:
                                                                precision    recall  f1-score   support

                                            ACCOUNT EXECUTIVE       0.00      0.00      0.00         1
                                                   ACCOUNTANT       0.58      0.88      0.70        25
                                          ACCOUNTS PAYABLE AP       0.00      0.00      0.00         1
                      ADMINISTRATOR OF INFORMATION TECHNOLOGY       0.00      0.00      0.00         1
                                    ADVENTURE RUN COORDINATOR       0.00      0.00      0.00         1
                                                     ADVOCATE       0.44      0.47      0.45        32
                                                  AGRICULTURE       0.75      0.23      0.35        13
                                               AIRCRAFT SALES       0.00      0.00      0.00         1
                  

/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter t

## Deep learning model

In [98]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping


max_len = 6000

# Removing the NaN values
df_resume_cleaned = df_resume.dropna(subset=['Resume', 'Category'])

X = df_resume_cleaned['Resume']
y = df_resume_cleaned['Category']

# Convert labels to integers
label_encoder = LabelEncoder()
y_labels = label_encoder.fit_transform(y)

# Tokenizer to convert text into sequences of integers
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

# Padding to ensure all sequences have the same length
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_seq, padding='post', maxlen=max_len)

# Create the model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dense(len(df_resume_cleaned['Category'].unique()), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
model.fit(X_padded, y_labels, epochs=1, validation_split=0.2, callbacks=[early_stopping])


# Evaluate the model on the entire dataset
test_loss, test_acc = model.evaluate(X_padded, y_labels)
print("Accuracy on the entire dataset:", test_acc)

new_line = {'model': "Neural Network model" , 'score': round(test_acc,2)}
results_df = results_df.append(new_line, ignore_index=True)


126/126 [==============================] - 122s 970ms/step - loss: 5.0796 - accuracy: 0.0299
Accuracy on the entire dataset: 0.029858173802495003


/var/folders/p9/_030_tzn0cx_x11mkmfz6w800000gn/T/ipykernel_37208/909535630.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_line, ignore_index=True)


## Pre Train models

In [103]:
# ! pip install transformers
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [106]:
# Removing the NaN values
df_resume_cleaned = df_resume.dropna(subset=['Resume', 'Category'])

X = df_resume_cleaned['Resume']
y = df_resume_cleaned['Category']


# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df_resume_cleaned['Resume'], df_resume_cleaned['Category'], test_size=0.2, random_state=42)

# Charger le tokenizer et le modèle BERT pré-entraîné
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df_resume_cleaned['Category'].unique()))

# Tokeniser les données
X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='tf')
X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='tf')

# Convertir BatchEncoding en un format que TensorFlow peut gérer
X_train_tokens_tf = {key: tf.convert_to_tensor(X_train_tokens[key]) for key in X_train_tokens}
X_test_tokens_tf = {key: tf.convert_to_tensor(X_test_tokens[key]) for key in X_test_tokens}

# Compiler le modèle
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Entraîner le modèle
model.fit(X_train_tokens_tf, y_train, epochs=3, batch_size=32)

# Évaluer sur les données de test
y_pred_probs = model.predict(X_test_tokens_tf)[0]
y_pred = tf.argmax(y_pred_probs, axis=1)

# Évaluation
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

new_line = {'model': "BERT" , 'score': round(accuracy,2)}
results_df = results_df.append(new_line, ignore_index=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


2024-01-16 21:51:16.805562: W tensorflow/core/framework/op_kernel.cc:1805] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'Cast_1' defined at (most recent call last):
    File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/Users/mathieubartozzi/.pyenv/versions/3.10.6/envs/lewagon/lib

# Results

In [100]:
results_df = results_df.sort_values(by='score', ascending=False)
results_df = results_df.reset_index(drop=True)
results_df

,model,score
0,LogisticRegression with TF-IDF,0.61
1,RandomForestClassifier with TF-IDF,0.58
2,Naive Baye Model,0.58
3,DummyClassifier with 'most_frequent' strategy,0.05
4,DummyClassifier with 'prior' strategy,0.05
5,DummyClassifier with 'stratified' strategy,0.03
6,Neural Network model,0.03
7,DummyClassifier with 'uniform' strategy,0.00
